In [31]:
import ROOT
import ROOT
import csv
import pandas as pd
import numpy as np

c = 3*10**8

In [32]:
file = ROOT.TFile("final_hlla_Z.root")

In [33]:
tree = file.Get('events')


In [34]:
branches = tree.GetListOfBranches()

for branch in branches:
    print(branch.GetName())

numParticles
eventweight
scale
alpha_qed
alpha_qcd
pid
status
mother1
mother2
color1
color2
px
py
pz
energy
mass
tau
spin


In [35]:
# From the data we conclude 
# Particle ID Higgs : 25
# Particle ID Z boson : 23
# Particle ID \gamma : 22
# Particle ID Mu+ : -13
# Particle ID Mu- : 13
# Particle ID e+ : -11
# Particle ID e- : 11

Tree 1 : Only Z Propagator

In [36]:

# Open a CSV file for writing
with open("(ZProp)DATA_particle_momentum.csv", mode="w", newline="") as csvfile:
    csvwriter = csv.writer(csvfile)

    # Write the header row
    csvwriter.writerow(["Event", "Particle", "Px", "Py", "Pz", "Energy", "Mass", "PID"])

    # Initialize event counter
    event_counter = 0

    # Loop over all events in the tree
    for event in tree:
        # Access branches for all particles in the event
        x = event.px
        y = event.py
        z = event.pz
        e = event.energy  # Energy
        m = event.mass    # Mass
        pid = event.pid   # Particle IDs

        # Loop over all particles in the event
        for j in range(len(pid)):
            particle_name = None

            # Determine the particle name based on PID
            if pid[j] == 25:
                particle_name = "Higgs"
            elif pid[j] == 23:
                particle_name = "Z Boson"
            elif pid[j] == 22:
                particle_name = "Photon"
            elif pid[j] == -13:
                particle_name = "Muon+"
            elif pid[j] == 13:
                particle_name = "Muon-"
            elif pid[j] == -11:
                particle_name = "Electron+"
            elif pid[j] == 11:
                particle_name = "Electron-"

            # If a recognized particle, write the data to the CSV
            if particle_name:
                csvwriter.writerow([event_counter, particle_name, x[j], y[j], z[j], e[j], m[j], pid[j]])

        # Increment event counter
        event_counter += 1

print("Particle momentum data saved to '(ZProp)DATA_particle_momentum.csv'.")


Particle momentum data saved to '(ZProp)DATA_particle_momentum.csv'.


In [37]:
import pandas as pd
import numpy as np

# Load the CSV file
input_file = "(ZProp)DATA_particle_momentum.csv"  # Replace with your file name
data = pd.read_csv(input_file)

# Output file for Lorentz transformed data
output_file = "Transformed_momentum.csv"

# Step 1: Define the Lorentz transformation matrix function
def lorentz_matrix(vx_c, vy_c, vz_c):
    """
    Create a Lorentz transformation matrix for a boost with velocity (vx_c, vy_c, vz_c).
    """
    beta = np.sqrt(vx_c**2 + vy_c**2 + vz_c**2)
    if beta >= 1:
        raise ValueError("Velocity magnitude must be less than the speed of light (beta < 1).")
    
    gamma = 1.0 / np.sqrt(1 - beta**2)
    
    # Lorentz matrix (4x4) structure
    L = np.zeros((4, 4))
    L[0, 0] = gamma
    L[1, 1] = 1 + (gamma**2)/(1+gamma) * vx_c**2
    L[2, 2] = 1 + (gamma**2)/(1+gamma) * vy_c**2
    L[3, 3] = 1 + (gamma**2)/(1+gamma) * vz_c**2
    L[1, 2] = L[2, 1] = (gamma**2)/(1+gamma) * vx_c * vy_c
    L[1, 3] = L[3, 1] = (gamma**2)/(1+gamma) * vx_c * vz_c
    L[2, 3] = L[3, 2] = (gamma**2)/(1+gamma) * vz_c * vy_c
    L[0, 1] = L[1, 0] = -gamma * vx_c
    L[0, 2] = L[2, 0] = -gamma * vy_c
    L[0, 3] = L[3, 0] = -gamma * vz_c
    
    return L

# Step 2: Initialize a list to store the transformed data
transformed_data = []


# Step 3: Loop over unique events
for event_id in data["Event"].unique():
    event_data = data[data["Event"] == event_id]

    # Extract Z boson data
    z_boson_data = event_data[event_data["Particle"] == "Z Boson"]

    if not z_boson_data.empty:
        # Get Z boson's momentum and energy
        z_boson_vector = z_boson_data[["Px", "Py", "Pz", "Energy"]].values[0]
        px, py, pz, energy = z_boson_vector

        # Compute velocity components (v_x_c, v_y_c, v_z_c)
        vx_c, vy_c, vz_c = px / energy, py / energy, pz / energy

        # Get the Lorentz transformation matrix
        L = lorentz_matrix(vx_c, vy_c, vz_c)

        # Step 4: Apply the Lorentz transformation to all particles in the event
        for _, particle in event_data.iterrows():
            # Original 4-momentum (E, Px, Py, Pz)
            original_vector = np.array([particle["Energy"], particle["Px"], particle["Py"], particle["Pz"]])

            # Apply Lorentz transformation
            transformed_vector = np.dot(L, original_vector)

            # Append transformed data
            transformed_data.append([
                event_id,
                particle["Particle"],
                transformed_vector[1],  # Px
                transformed_vector[2],  # Py
                transformed_vector[3],  # Pz
                transformed_vector[0],  # Energy
                particle["Mass"],
                particle["PID"]
            ])

# Step 5: Create a new DataFrame and save it
transformed_df = pd.DataFrame(
    transformed_data,
    columns=["Event", "Particle", "Px", "Py", "Pz", "Energy", "Mass", "PID"]
)

# Save the transformed data to a CSV file
transformed_df.to_csv(output_file, index=False)

print(f"Lorentz transformed data saved to '{output_file}'.")

Lorentz transformed data saved to 'Transformed_momentum.csv'.


In [38]:
import pandas as pd

# Input file containing the particle data
input_file = "(ZProp)DATA_particle_momentum.csv"  # Replace with your actual file name

# Read the data into a pandas dataframe
data = pd.read_csv(input_file)

# Define a function to calculate velocity components
def calculate_velocity(row):
    # Extract momentum components and energy
    px, py, pz, energy = row['Px'], row['Py'], row['Pz'], row['Energy']
    
    # Calculate velocity components
    vx = px / energy
    vy = py / energy
    vz = pz / energy
    
    return pd.Series([vx, vy, vz], index=['Vx', 'Vy', 'Vz'])

# Apply the function to each row in the dataframe and create new columns for velocities
data[['Vx', 'Vy', 'Vz']] = data.apply(calculate_velocity, axis=1)

# Output the new dataframe to a CSV file
output_file = "particles_velocities.csv"  # Replace with your desired output file name
data.to_csv(output_file, index=False)

print(f"Velocity data saved to {output_file}")


Velocity data saved to particles_velocities.csv


In [39]:
import pandas as pd

# Input file containing the particle data
input_file = "Transformed_momentum.csv"  # Replace with your actual file name

# Read the data into a pandas dataframe
data = pd.read_csv(input_file)

# Define a function to calculate velocity components
def calculate_velocity(row):
    # Extract momentum components and energy
    px, py, pz, energy = row['Px'], row['Py'], row['Pz'], row['Energy']
    
    # Calculate velocity components
    vx = px / energy
    vy = py / energy
    vz = pz / energy
    
    return pd.Series([vx, vy, vz], index=['Vx', 'Vy', 'Vz'])

# Apply the function to each row in the dataframe and create new columns for velocities
data[['Vx', 'Vy', 'Vz']] = data.apply(calculate_velocity, axis=1)

# Output the new dataframe to a CSV file
output_file = "Transformed_velocities.csv"  # Replace with your desired output file name
data.to_csv(output_file, index=False)

print(f"Velocity data saved to {output_file}")


Velocity data saved to Transformed_velocities.csv
